## Pratham Agrawal - Camera Calibration task for Internship

In [59]:
import cv2
import glob
import copy
import math
import numpy as np
import imageio
import matplotlib.pyplot as plt

from pprint import pprint as pp

In [60]:
def load_images(filenames):
    return [imageio.imread(filename) for filename in filenames]

### The dataset is acquired from the link given, the photos are clicked through phone's camera and uploaded to kaggle for its better IDE. 

the link for the dataset made: https://www.kaggle.com/datasets/fearsomejockey/wsawdq

In [61]:
filenames = list(sorted(glob.glob("../input/wsawdq/*.jpg")))
imgs = load_images(filenames)

In [62]:

corners = [cv2.findChessboardCorners(i, (6,8)) for i in imgs]

In [63]:

corners2 = copy.deepcopy(corners)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)

imgs_grey = [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in imgs]

cornersRefined = [cv2.cornerSubPix(i, cor[1], (8, 6), (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_grey, corners2)]


In [64]:
imgs2 = copy.deepcopy(imgs)

tmp = [cv2.drawChessboardCorners(img, (8,6), cor[1], cor[0]) for img, cor in zip(imgs2, corners) if cor[0]]

In [65]:
# Original Image
plt.figure()
plt.imshow(imgs2[0])

In [66]:
# Image with the corners drawed
plt.figure()
plt.imshow(imgs2[2])

In [67]:
def get_chessboard_points(chessboard_shape, dx, dy):
    return [[(i%chessboard_shape[0])*dx, (i//chessboard_shape[0])*dy, 0] for i in range(np.prod(chessboard_shape))]

cb_points = get_chessboard_points((6, 8), 30, 30)
# pp(cb_points)

In [68]:

valid_corners = [cor[1] for cor in corners if cor[0]]

num_valid_images = len(valid_corners)


real_points = get_chessboard_points((8, 6), 30, 30)


object_points = np.asarray([real_points for i in range(num_valid_images)], dtype=np.float32)


image_points = np.asarray(valid_corners, dtype=np.float32)


rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, imgs[1].shape[0:2], None, None)

extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))

np.savez('calib_left', intrinsic=intrinsics, extrinsic=extrinsics)


In [69]:
# Displayig required output 
print(" Camera matrix:") 
print(intrinsics) 
  
print("\n Distortion coefficient:") 
print(dist_coeffs) 
  
print("\n Rotation Vectors:") 
print(rvecs) 
  
print("\n Translation Vectors:") 
print(tvecs)

In [70]:
v1 = np.matmul(np.linalg.inv(intrinsics), [10,10,1])
v2 = np.matmul(np.linalg.inv(intrinsics), [60,60,1])

pp("Diagonal FOV: {} degrees".format(math.degrees(py_ang(v1,v2))))

### The parameters such as Diagonal FOV, camera matrix, rotation vectors, translation vectors, and distortion coefficient matrix are shown. 